In [1]:
import jinja2

In [2]:
TauDEM_TEMPLATE = """
{%- for library in cls.libraries %}
import {{ library }}
{%- endfor %}

def {{ cls.method1 }}({{ cls.parameter1}}):
    {%- for (name, prop) in cls.property1.items() %}
    {{ name }} = {{ prop }}
    {%- endfor %}
    print(output)

{%- for (name, cmd) in cls.method2.items() %}

def {{ name }}({{ cmd[0] }}):
    {{ cmd[2] }}
    {{ name }} = {{ cmd[1] }}
    return {{ name }}
    {%- endfor %} 
"""

In [3]:
TauDEM_Attributes_JSON = {
    "library": ["os", "subprocess"],
    "method1": "taudem_execute",
    "paramter1":["taudem_cmd"],
    "object1": {"p": "subprocess.Popen(taudem_cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)", 
                "output": "p.communicate()[0].decode('utf-8')"},
    "method2": {"pitremove":['np, input, output',
                             '"mpirun -np {} pitremove -z {} -fel {}".format(np, input, output)',
                             '""" command: pitremove -z dem.tif -fel demfel.tif, demfile: input elevation grid, felfile: output elevations with pits filled """'  
                             ],
                 "d8flowdir":['np, input, output1, output2',
                              '"mpirun -np {} d8flowdir -fel {} -p {} -sd8 {}".format(np, input, output1, output2)',
                              '""" command: d8flowdir -fel demfel.tif -p demp.tif -sd8 demsd8.tif, demfile: Pit filled elevation input data, pointfile: D8 flow directions output, slopefile: D8 slopes output """'
                              ],
                 "dinfflowdir":['np, input, output1, output2',
                                '"mpirun -np {} dinfflowdir -fel {} -ang {} -slp {}".format(np, input, output1, output2)',
                                '""" command: dinfflowdir -fel demfel.tif -ang demang.tif -slp demslp.tif, demfile: Pit filled elevation input data, angfile: Dinf flow directions output, slopefile: Dinf slopes output """'
                              ],
                 "aread8":['np, input, output',
                           '"mpirun -np {} aread8 -p {} -ad8 {}".format(np, input, output)',
                           '""" command: aread8 -p demp.tif -ad8 demad8.tif, pfile: input flow directions grid, ad8file: output contributing area grid """'
                              ],
                 "areadinf":['np, input, output',
                           '"mpirun -np {} areadinf -ang {} -sca {}}".format(np, input, output)',
                           '""" command: areadinf -ang demang.tif -sca demsca.tif, angfile: Dinf angles input file, scafile: Dinf contributing areas output file """'
                              ],
}
}

In [4]:
class JsonToSimApi(object):
    
    simulation_template = TauDEM_TEMPLATE
    
    def __init__(self, json):
        self.json = json
        self.val_list = []
        
    @property
    def libraries(self):
        return list(self.json["library"]) 
            
    @property
    def method1(self):
        return self.json['method1']

    @property
    def method2(self):
        return {key: val for key, val in self.json["method2"].items()}
    
    @property
    def parameter1(self):
        return ', '.join(self.json["paramter1"])
    
    @property
    def parameter2(self):
        return list(', '.join(self.json["paramter2"]))

    @property
    def property1(self):
        return {key: val for key, val in self.json["object1"].items()}

    @property
    def property2(self):
        return {key: val for key, val in self.json["object2"].items()}    

    def object_code(self):
        return jinja2.Template(self.simulation_template).render(cls=self)

In [5]:
code = JsonToSimApi(TauDEM_Attributes_JSON).object_code()
print(code)


import os
import subprocess

def taudem_execute(taudem_cmd):
    p = subprocess.Popen(taudem_cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    output = p.communicate()[0].decode('utf-8')
    print(output)

def pitremove(np, input, output):
    """ command: pitremove -z dem.tif -fel demfel.tif, demfile: input elevation grid, felfile: output elevations with pits filled """
    pitremove = "mpirun -np {} pitremove -z {} -fel {}".format(np, input, output)
    return pitremove

def d8flowdir(np, input, output1, output2):
    """ command: d8flowdir -fel demfel.tif -p demp.tif -sd8 demsd8.tif, demfile: Pit filled elevation input data, pointfile: D8 flow directions output, slopefile: D8 slopes output """
    d8flowdir = "mpirun -np {} d8flowdir -fel {} -p {} -sd8 {}".format(np, input, output1, output2)
    return d8flowdir

def dinfflowdir(np, input, output1, output2):
    """ command: dinfflowdir -fel demfel.tif -ang demang.tif -slp demslp.tif, demfile: Pit filled elevatio

In [6]:
%%capture cap --no-stderr
print(code)

In [7]:
%%capture cap --no-stderr
with open('taudem_test.py', 'w') as f:
    f.write(cap.stdout)

In [8]:
!autopep8 --in-place --aggressive --aggressive 'taudem_test.py'

In [9]:
import shutil
shutil.copy('taudem_test.py', 'taudem.py')

'taudem.py'